In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_matching_tools.io.graph_dataset import GraphDataset
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
from graph_matching_tools.algorithms.multiway.hippi import hippi_multiway_matching
from graph_matching_tools.algorithms.kernels.gaussian import create_gaussian_node_kernel
from graph_matching_tools.algorithms.kernels.utils import create_full_node_affinity_matrix
import scipy.io as sio
# from KerGM_python.grackel import *
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/KerGM_python/grackel/")
import re
import KerGM_python.grackel.grackelpy.algorithms.matching as match
import KerGM_python.grackel.grackelpy.algorithms.multimatch as mm
import KerGM_python.grackel.grackelpy.algorithms.quickmatch as qm
import KerGM_python.grackel.grackelpy.utils.measures as measures
import KerGM_python.grackel.grackelpy.utils.kernels as kern
import KerGM_python.grackel.grackelpy.utils.utils as utils
import KerGM_python.grackel.graph_processing.graph_processing as gp

In [2]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [3]:
def create_full_adjacency_matrix(graphs):
    """
    Create the full adjacency matrix with the matrices on the diagonal

    :param list graphs: the list of graphs
    :return:
    """
    sizes = []
    full_size = 0
    for g in graphs:
        size = nx.number_of_nodes(g)
        sizes.append(size)
        full_size += size

    a = np.zeros((full_size, full_size))

    index = 0
    for i in range(len(graphs)):
        adj = (nx.adjacency_matrix(graphs[i])).toarray()
        a[index:index+sizes[i], index:index+sizes[i]] = adj
        index += sizes[i]

    return a, sizes

In [4]:
path_to_graph_folder = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/test/'

In [5]:
trials = np.sort(os.listdir(path_to_graph_folder))

sigma = 200
u_dim = 30 # dim for universe of nodes

scores = {100:[],400:[],700:[],1000:[],1300:[]}

for trial in trials:
    
    if float(trial) >= 0.0:
        
        print('trial: ', trial)

        all_files = os.listdir(path_to_graph_folder+trial)

        for folder in all_files:

            if os.path.isdir(path_to_graph_folder+trial+'/'+ folder):

                print('Noise folder: ',folder)

                path_to_graphs = path_to_graph_folder + '/' + trial + '/' + folder+'/graphs/'
                path_to_groundtruth_ref = path_to_graph_folder + '/' + trial +'/' + folder + '/permutation_to_ref_graph.gpickle'
                path_to_groundtruth  = path_to_graph_folder + '/' + trial + '/' + folder + '/ground_truth.gpickle'


                noise = folder.split(',')[0].split('_')[1]
                graph_meta = GraphDataset(path_to_graphs, path_to_groundtruth_ref)
                ground_truth =  nx.read_gpickle(path_to_groundtruth)   
                res = get_permutation_matrix_from_dictionary(ground_truth, graph_meta.sizes)
                

                node_kernel = create_gaussian_node_kernel(sigma,'coord')
                knode = create_full_node_affinity_matrix(graph_meta.list_graphs, node_kernel)
                #s = np.eye(knode.shape[0], knode.shape[1])
                path_init = path_to_graph_folder + '/'+ trial + '/' + folder + '/X_kmeans.mat'                
                X_kmeans = sio.loadmat(path_init)
                #u = X_kmeans['U']
                s = X_kmeans['full_assignment_mat']

                # Old Implementation
#                 node_kernel = kern.create_node_kernel(sigma, "coord")   #Change coord to sphere_3dcoords for real data
#                 knode = mm.create_full_node_affinity_matrix(graph_meta.list_graphs, node_kernel)
#                 s, _ = utils.create_full_adjacency_matrix(graph_meta.list_graphs)
#                 s = np.eye(knode.shape[0], knode.shape[1])
#                 u = mm.hippi_multiway_matching(s, graph_meta.sizes, knode, u_dim)
#                 P = u @ u.T

                u = np.ones((s.shape[0], u_dim)) / u_dim + 1e-3 * np.random.randn(s.shape[0], u_dim)
    
            
                U = hippi_multiway_matching(s, graph_meta.sizes, knode, u_dim)
                P = U @ U.T

                HiPPi_X = {}
                HiPPi_X['full_assignment_mat'] = P
                
                print('P shape', P.shape)
                print('s shape', s.shape)

                sio.savemat(path_to_graph_folder + '/' + trial + '/' + folder + '/X_HiPPi.mat',HiPPi_X)

                f1, prec, rec = matching.compute_f1score(P,res)

                scores[int(noise)].append(f1)

trial:  0
Noise folder:  noise_100,outliers_varied
P shape (371, 371)
s shape (371, 371)
Noise folder:  noise_400,outliers_varied
P shape (386, 386)
s shape (386, 386)
Noise folder:  noise_700,outliers_varied
P shape (331, 331)
s shape (331, 331)
Noise folder:  noise_1300,outliers_varied
P shape (371, 371)
s shape (371, 371)
Noise folder:  noise_1000,outliers_varied
P shape (391, 391)
s shape (391, 391)
trial:  1
Noise folder:  noise_100,outliers_varied
P shape (399, 399)
s shape (399, 399)
Noise folder:  noise_400,outliers_varied
P shape (376, 376)
s shape (376, 376)
Noise folder:  noise_700,outliers_varied
P shape (373, 373)
s shape (373, 373)
Noise folder:  noise_1300,outliers_varied
P shape (346, 346)
s shape (346, 346)
Noise folder:  noise_1000,outliers_varied
P shape (356, 356)
s shape (356, 356)
trial:  2
Noise folder:  noise_100,outliers_varied
P shape (374, 374)
s shape (374, 374)
Noise folder:  noise_400,outliers_varied
P shape (406, 406)
s shape (406, 406)
Noise folder:  noi

In [7]:
scores

{100: [0.15200918484500572,
  0.14778947368421053,
  0.16609783845278728,
  0.16381933068102036],
 400: [0.1680327868852459,
  0.18726591760299627,
  0.184394250513347,
  0.13416815742397137],
 700: [0.14322916666666666,
  0.13772860690900882,
  0.15698924731182798,
  0.19256110520722636],
 1000: [0.2282157676348548,
  0.1827776445190693,
  0.17051108095884213,
  0.1598818814596077],
 1300: [0.152885443583118,
  0.14711033274956214,
  0.1860873381610709,
  0.1976301311891663]}

In [ ]:
# nx.write_gpickle(scores,'Hippi_score.gpickle')

In [ ]:
# np.ones((s.shape[0], u_dim)) / u_dim